In [ ]:
import pandas as pd
import json
from collections import defaultdict
import re
from utilities import Tiempo
import dataExtraction
import statistics

In [ ]:
def extraerTiemposEntreInteracciones(rawData):
    tiempoInteracciones = defaultdict()
    ultimaInteraccionJugador = defaultdict()
    tiempoInteraccionesJugador = defaultdict(defaultdict)
    
    tiempoInicio_Interaccion = defaultdict()
    tiempoInteraccion_Completado = defaultdict()
    tiempoInteraccion_Reiniciado = defaultdict()
    
    tiempoInicio_Interaccion_Jugador = defaultdict(defaultdict)
    tiempoInteraccion_Completado_Jugador = defaultdict(defaultdict)
    tiempoInteraccion_Reiniciado_Jugador = defaultdict(defaultdict)
    
    anomalias = []
    
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erLevelExitButton = re.compile(r'\blevel_exit_button$\b') 
    erInitialized = re.compile(r'\binitialized$\b')
    erSeriousGame = re.compile(r'\bserious-game$\b')    
    erCompleted = re.compile(r'\bcompleted$\b')
    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
        
    for e in rawData:
        verb = e["verb"]["id"]
        obj = e["object"]["definition"]["type"]
        objectId = e["object"]["id"]
        name = e["actor"]["name"]
        
        levelCode = erIdLevel.split(objectId)[-1]
        if(levelCode != "editor_level"):
            
            if erSeriousGame.search(obj) and erInitialized.search(verb): #Si inicia el juego se reinicia su ultima interaccion
                if name in ultimaInteraccionJugador:
                    del ultimaInteraccionJugador[name]

            elif erLevel.search(obj):
                if erInitialized.search(verb): #Significa que ha iniciado o reiniciado el nivel
                    timestamp = e["timestamp"]

                    if "result" in e: #Inicia el nivel desde el menu
                        ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "inicio"}
                    else: #Reinicia el nivel
                        try:
                            t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                            #Añadimos la dif de tiempo al diccionario con todos los tiempos
                            if levelCode in tiempoInteracciones:                            
                                tiempoInteracciones[levelCode].append(t)
                            else:
                                tiempoInteracciones[levelCode] = [t]

                            #Añadimos la dif de tiempo al diccionario del jugador
                            if levelCode in tiempoInteraccionesJugador[name]:
                                tiempoInteraccionesJugador[name][levelCode].append(t)
                            else:
                                tiempoInteraccionesJugador[name][levelCode] = [t]
                            #Actualizamos ultima interaccion
                            ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "reinicio"}
                        except:
                            #Si entra aqui es porque ha reiniciado un nivel que no habia iniciado
                            #Trazas estan mal
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha reiniciado un nivel que no estaba iniciado"})

                elif erCompleted.search(verb) and e["result"]["score"]["raw"] != 0:
                    #Si completa un nivel o sale al menu se reinicia su ultima interaccion, si falla el intento pero vuelve a intentarlo no se reinicia
                    timestamp = e["timestamp"]
                    try:
                        t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)                        
                        #Añadimos la dif de tiempo al diccionario con todos los tiempos
                        if levelCode in tiempoInteracciones:                            
                            tiempoInteracciones[levelCode].append(t)
                        else:
                            tiempoInteracciones[levelCode] = [t]

                        #Añadimos la dif de tiempo al diccionario del jugador
                        if levelCode in tiempoInteraccionesJugador[name]:
                            tiempoInteraccionesJugador[name][levelCode].append(t)
                        else:
                            tiempoInteraccionesJugador[name][levelCode] = [t] 
                        #Actualizamos ultima interaccion, la borramos
                        del ultimaInteraccionJugador[name]
                    except:
                        #Si entra aqui es porque se ha completado un nivel que no se habia iniciado
                        #Probablemente por anomalias en las trazas
                        anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo": "Se ha completado un nivel que no estaba iniciado"})

            elif erGameObject.search(obj) and erInteracted.search(verb):
                if "result" in e and "extensions" in e["result"] and "level" in e["result"]["extensions"]:
                    levelCode = e["result"]["extensions"]["level"]
                    if not erLevelExitButton.search(objectId) and levelCode != "editor_level":
                        timestamp = e["timestamp"]
                        if name in ultimaInteraccionJugador:
                            #e["result"]["extensions"]["action"] <-- Que accion realiza
                            t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                            #Añadimos la dif de tiempo al diccionario con todos los tiempos
                            if levelCode in tiempoInteracciones:                            
                                tiempoInteracciones[levelCode].append(t)
                            else:
                                tiempoInteracciones[levelCode] = [t]

                            #Añadimos la dif de tiempo al diccionario del jugador
                            if levelCode in tiempoInteraccionesJugador[name]:
                                tiempoInteraccionesJugador[name][levelCode].append(t)
                            else:
                                tiempoInteraccionesJugador[name][levelCode] = [t]
                            ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "interaccion"}
                            
                        else:
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha interactuado en un nivel que no estaba iniciado"})
                        
                    
    return {"tiempoEntreInteracciones" : tiempoInteracciones, "tiempoEntreInteraccionesJugador" : tiempoInteraccionesJugador, "anomalias" : anomalias}

In [ ]:
def calcularMediaInteraccionesPorNivel(tiemposInteracciones):
    tiempoMedioInteracciones = defaultdict()
    
    for level in tiemposInteracciones:
        acumulador = Tiempo("0s")
        for t in tiemposInteracciones[level]:
            acumulador += t
        tiempoMedioInteracciones[level] = (acumulador / len(tiemposInteracciones[level]))
    return tiempoMedioInteracciones

In [ ]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [ ]:
JSONFile = open('trazasOrdenadas.json')
rawData = json.load(JSONFile)
#verTrazas(rawData)
result = extraerTiemposEntreInteracciones(rawData)
print(result["tiempoEntreInteracciones"])
JSONFile.close()
tiempoMedioInteracciones = calcularMediaInteraccionesPorNivel(result["tiempoEntreInteracciones"])
pd.DataFrame.from_dict(tiempoMedioInteracciones, orient='index').transpose()

In [ ]:
pd.DataFrame(result["anomalias"])
#Se ha interactuado en un nivel que no estaba iniciado -> Probablemente sea porque completa el nivel, minimiza la ventana de siguiente nivel o menu y se pone a mover las tarjetas

In [ ]:
def verTrazas(rawData):
    for e in rawData:
        name = e["actor"]["name"]
        if(name == "auvqa"):
            print(e)
            print("--------")